<a href="https://colab.research.google.com/github/truongthuanr/self-project/blob/main/06_QA_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Information

Reference: 
* [End-To-End Memory Networks](https://arxiv.org/pdf/1503.08895.pdf)

# import data

In [3]:
import numpy as np
import pickle

In [4]:
with open("/content/drive/MyDrive/01_Personal/01_Study/01_ML/04_QAChatBot/train_qa.txt", "rb") as f:
  train_data = pickle.load(f)

with open("/content/drive/MyDrive/01_Personal/01_Study/01_ML/04_QAChatBot/test_qa.txt", "rb") as f:
  test_data = pickle.load(f)

In [5]:
print(train_data[506][0])
print(train_data[506][1])
print(train_data[506][2])


['Daniel', 'took', 'the', 'milk', 'there', '.', 'Sandra', 'journeyed', 'to', 'the', 'bedroom', '.', 'Daniel', 'discarded', 'the', 'milk', '.', 'Mary', 'went', 'to', 'the', 'bathroom', '.']
['Is', 'Mary', 'in', 'the', 'bathroom', '?']
yes


In [6]:
all_data = train_data + test_data

In [7]:
len(train_data), len(test_data), len(all_data)

(10000, 1000, 11000)

In [8]:
vocab = set()

for story, question, answer in all_data:
  vocab = vocab.union(set(story))
  vocab = vocab.union(set(question))

In [9]:
vocab.add('no')
vocab.add('yes')

In [10]:
len(vocab)

37

In [11]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [12]:
all_story_lens = [len(data[0]) for data in all_data]
max_story_len = max(all_story_lens)
max_story_len

156

In [13]:
all_ques_lens = [len(data[1]) for data in all_data]
max_ques_len = max(all_ques_lens)
max_ques_len

6

# Vetorize data

In [14]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [15]:
tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

In [16]:
tokenizer.word_index

{'.': 34,
 '?': 18,
 'apple': 11,
 'back': 20,
 'bathroom': 10,
 'bedroom': 12,
 'daniel': 6,
 'discarded': 33,
 'down': 32,
 'dropped': 30,
 'football': 7,
 'garden': 29,
 'got': 22,
 'grabbed': 26,
 'hallway': 36,
 'in': 4,
 'is': 14,
 'john': 25,
 'journeyed': 16,
 'kitchen': 2,
 'left': 27,
 'mary': 24,
 'milk': 8,
 'moved': 19,
 'no': 37,
 'office': 17,
 'picked': 1,
 'put': 13,
 'sandra': 5,
 'the': 3,
 'there': 21,
 'to': 35,
 'took': 15,
 'travelled': 23,
 'up': 28,
 'went': 9,
 'yes': 31}

In [17]:
train_story_text = []
train_ques_text = []
train_ans_text = []

In [18]:
for story,ques,ans in train_data:
  train_story_text.append(story)
  train_ques_text.append(ques)
  train_ans_text.append(ans)

In [19]:
train_story_text[0]

['Mary',
 'moved',
 'to',
 'the',
 'bathroom',
 '.',
 'Sandra',
 'journeyed',
 'to',
 'the',
 'bedroom',
 '.']

In [20]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [21]:
len(train_story_seq)

10000

In [22]:
train_story_seq[0]

[24, 19, 35, 3, 10, 34, 5, 16, 35, 3, 12, 34]

# Vectorization

In [48]:
def vectorize_stories(data, word_index=tokenizer.word_index, max_story_len=max_story_len,max_question_len=max_ques_len):
  '''
  INPUT: 
  
  data: consisting of Stories,Queries,and Answers
  word_index: word index dictionary from tokenizer
  max_story_len: the length of the longest story (used for pad_sequences function)
  max_question_len: length of the longest question (used for pad_sequences function)


  OUTPUT:
  
  Vectorizes the stories,questions, and answers into padded sequences. We first loop for every story, query , and
  answer in the data. Then we convert the raw words to an word index value. Then we append each set to their appropriate
  output list. Then once we have converted the words to numbers, we pad the sequences so they are all of equal length.
  
  Returns this in the form of a tuple (X,Xq,Y) (padded based on max lengths)
  '''
  
  
  # X = STORIES
  X = []
  # Xq = QUERY/QUESTION
  Xq = []
  # Y = CORRECT ANSWER
  Y = []
  
  
  for story, query, answer in data:
      
      # Grab the word index for every word in story
      x = [word_index[word.lower()] for word in story]
      # Grab the word index for every word in query
      xq = [word_index[word.lower()] for word in query]
      
      # Grab the Answers (either Yes/No so we don't need to use list comprehension here)
      # Index 0 is reserved so we're going to use + 1
      y = np.zeros(len(word_index) + 1)
      
      # Now that y is all zeros and we know its just Yes/No , we can use numpy logic to create this assignment
      #
      y[word_index[answer]] = 1
      
      # Append each set of story,query, and answer to their respective holding lists
      X.append(x)
      Xq.append(xq)
      Y.append(y)
      
  # Finally, pad the sequences based on their max length so the RNN can be trained on uniformly long sequences.
      
  # RETURN TUPLE FOR UNPACKING
  return (pad_sequences(X, maxlen=max_story_len),pad_sequences(Xq, maxlen=max_question_len), np.array(Y))

In [49]:
inputs_train, queries_train, answers_train = vectorize_stories(train_data)

In [52]:
inputs_test, queries_test, answers_test = vectorize_stories(test_data)

In [53]:
inputs_test[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0, 24, 22,  3,  8, 21, 34, 25, 19, 35,
        3, 12, 34], dtype=int32)

In [54]:
inputs_train

array([[ 0,  0,  0, ...,  3, 12, 34],
       [ 0,  0,  0, ...,  3, 36, 34],
       [ 0,  0,  0, ...,  3, 10, 34],
       ...,
       [ 0,  0,  0, ...,  3, 12, 34],
       [ 0,  0,  0, ...,  8, 21, 34],
       [ 0,  0,  0, ..., 11, 21, 34]], dtype=int32)

In [56]:
inputs_train.shape

(10000, 156)

In [55]:
answers_test

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [57]:
answers_test.shape

(1000, 38)

In [58]:
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout
from keras.layers import add, dot, concatenate
from keras.layers import LSTM

Recall we technically have two inputs, stories and questions. So we need to use placeholders. `Input()` is used to instantiate a Keras tensor.

In [60]:
input_sequence = Input((max_story_len,))
question = Input((max_ques_len,))

In [61]:
vocab_size = len(vocab) + 1

In [62]:
# Input gets embedded to a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,output_dim=64))
input_encoder_m.add(Dropout(0.3))

In [63]:
(samples, story_maxlen, embedding_dim)

NameError: ignored